In [ ]:
from transformers import GPT2TokenizerFast
new_tokenizer = GPT2TokenizerFast.from_pretrained("openai-community/gpt2", cache_dir="hfcache")

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
ds = load_dataset("facebook/belebele", "zho_Hans", cache_dir="hfcache", split="test")

Using the latest cached version of the dataset since facebook/belebele couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'zho_Hans' at hfcache/facebook___belebele/zho_Hans/0.0.0/7899cdfa4e1e0d733fd77c848e2c273cb1d32be2 (last modified on Tue Nov 26 19:18:05 2024).


In [3]:
ds2=ds.rename_column('flores_passage', 'text')

In [4]:
new_tokenizer.model_max_length=512

from coach import Coach

def tokenize_function(example):
    return new_tokenizer(example["text"], example["text"], truncation=True)

tokenized_datasets = ds2.map(tokenize_function, batched=True)

coach = Coach(tokenized_datasets, new_tokenizer)


In [5]:
modified_dataset = tokenized_datasets.map(coach)
# coach(tokenized_datasets[0])

In [6]:
modified_dataset[0]

{'link': 'https://en.wikibooks.org/wiki/Accordion/Right_hand',
 'question_number': 1,
 'text': '确保你的手尽可能地放松，同时仍可以正确地敲击所有的音符—手指也尽量不要做太多无关的动作。 这样就能让你尽可能节省体力。记住，不用像在钢琴上那样用力敲击琴键来提升音量。 为了提高手风琴的音量，可以更用力或以更快的速度拉风箱。',
 'question': '根据这段文字，以下哪项不会被认为是成功演奏手风琴的准确技巧？',
 'mc_answer1': '用力敲击琴键来提升音量',
 'mc_answer2': '尽量减少不必要的动作来节省体力',
 'mc_answer3': '在手部放松的同时注意正确敲击音符',
 'mc_answer4': '以更快的速度拉风箱来提升音量',
 'correct_answer_num': '1',
 'dialect': 'zho_Hans',
 'ds': datetime.datetime(2023, 7, 21, 0, 0),
 'input_ids': [163,
  94,
  106,
  46479,
  251,
  19526,
  254,
  21410,
  33699,
  233,
  22887,
  121,
  20998,
  107,
  47797,
  121,
  28839,
  108,
  162,
  242,
  122,
  30266,
  122,
  171,
  120,
  234,
  28938,
  234,
  33768,
  114,
  20015,
  235,
  20998,
  107,
  20015,
  98,
  29826,
  96,
  163,
  94,
  106,
  28839,
  108,
  46763,
  110,
  49035,
  119,
  33699,
  222,
  17312,
  231,
  21410,
  165,
  253,
  111,
  163,
  105,
  99,
  960,
  33699,
  233,
  162,
  234,
  229,
  20046,
  253